In [1]:
import pandas as pd
import numpy as np
import re

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam

from keras.utils import np_utils

import warnings 
warnings.filterwarnings(action='ignore')

In [2]:
PATH = ''

In [3]:
train = pd.read_csv(PATH + "train_data.csv")
test = pd.read_csv(PATH + "test_data.csv")
submission = pd.read_csv(PATH + "sample_submission.csv")


## Vectorization

In [4]:
# 먼저 train 데이터와 test 데이터 인덱스 없이 배열로 만들기
X_train = np.array([x for x in train['title']])
X_test = np.array([x for x in test['title']])
Y_train = np.array([x for x in train['topic_idx']])

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)

(45654,)
(9131,)
(45654,)


In [5]:
# Tokenizer
from keras.preprocessing.text import Tokenizer
vocab_size = 2000  

tokenizer = Tokenizer(num_words = vocab_size)  
  # Tokenizer 는 데이터에 출현하는 모든 단어의 개수를 세고 빈도 수로 정렬해서 
  # num_words 에 지정된 만큼만 숫자로 반환하고, 나머지는 0 으로 반환합니다                 
tokenizer.fit_on_texts(X_train) # Tokenizer 에 데이터 실제로 입력
sequences_train = tokenizer.texts_to_sequences(X_train)    # 문장 내 모든 단어를 시퀀스 번호로 변환
sequences_test = tokenizer.texts_to_sequences(X_test)      # 문장 내 모든 단어를 시퀀스 번호로 변환

print(len(sequences_train), len(sequences_test))

45654 9131


## 임베딩

In [6]:
# 변환된 시퀀스 번호를 이용해 단어 임베딩 벡터 생성
word_index = tokenizer.word_index

In [7]:
# 독립변수 데이터 전처리
  ## 문장의 길이기 제각각이기 때문에 벡터 크기 다 다름
  ## 그러므로 최대 시퀀스 길이 크기(211) 만큼 넉넉하게 늘리고
  ## 패딩(padding) 작업을 통해 나머지 빈 공간을 0으로 채움
max_length = 14    # 위에서 그래프 확인 후 정함
padding_type='post'

train_x = pad_sequences(sequences_train, padding='post', maxlen=max_length)
test_x = pad_sequences(sequences_test, padding=padding_type, maxlen=max_length)

print(train_x.shape, test_x.shape)

(45654, 14) (9131, 14)


In [8]:
# 종속변수 데이터 전처리
train_y = np_utils.to_categorical(Y_train) # Y_train 에 원-핫 인코딩
print(train_y)
print(train_y.shape)

[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
(45654, 7)


## 모델링

In [12]:
#파라미터 설정
vocab_size = 2000 # 제일 많이 사용하는 사이즈
embedding_dim = 200  
max_length = 14    # 위에서 그래프 확인 후 정함
padding_type='post'

In [13]:
# LSTM 레이어를 사용한 모델 (model2) 정의
model2 = Sequential([Embedding(vocab_size, embedding_dim, input_length =max_length),
        tf.keras.layers.Bidirectional(LSTM(units = 64, return_sequences = True)),
        tf.keras.layers.Bidirectional(LSTM(units = 64, return_sequences = True)),
        tf.keras.layers.Bidirectional(LSTM(units = 64)),
        Dense(7, activation='softmax')    # 결과값이 0~4 이므로 Dense(5)
    ])
    
model2.compile(loss= 'categorical_crossentropy', #여러개 정답 중 하나 맞추는 문제이므로 손실 함수는 categorical_crossentropy
              optimizer= 'adam',
              metrics = ['accuracy']) 
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 14, 200)           400000    
_________________________________________________________________
lstm (LSTM)                  (None, 14, 64)            67840     
_________________________________________________________________
lstm_1 (LSTM)                (None, 14, 64)            33024     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_1 (Dense)              (None, 7)                 455       
Total params: 534,343
Trainable params: 534,343
Non-trainable params: 0
_________________________________________________________________


In [14]:
# 모델 실행해보기
history = model2.fit(train_x, train_y, epochs=10, batch_size=100, validation_split= 0.2) 
  # 데이터가 50000개가 넘어 학습시간이 오래 걸리기 때문에 batch size 를 100으로 크게 잡았다
  # 추이를 확인하게 위해 일단 10회만 시도

Epoch 1/10
366/366 [==============================] - 33s 76ms/step - loss: 1.1099 - accuracy: 0.5889 - val_loss: 1.1619 - val_accuracy: 0.5925
Epoch 2/10
366/366 [==============================] - 27s 74ms/step - loss: 0.8126 - accuracy: 0.7192 - val_loss: 1.1608 - val_accuracy: 0.5768
Epoch 3/10
366/366 [==============================] - 27s 74ms/step - loss: 0.7633 - accuracy: 0.7399 - val_loss: 1.1802 - val_accuracy: 0.5681
Epoch 4/10
366/366 [==============================] - 27s 75ms/step - loss: 0.7331 - accuracy: 0.7480 - val_loss: 1.1730 - val_accuracy: 0.5716
Epoch 5/10
366/366 [==============================] - 27s 74ms/step - loss: 0.7123 - accuracy: 0.7530 - val_loss: 1.1003 - val_accuracy: 0.5979
Epoch 6/10
366/366 [==============================] - 29s 79ms/step - loss: 0.6908 - accuracy: 0.7591 - val_loss: 1.1719 - val_accuracy: 0.5745
Epoch 7/10
366/366 [==============================] - 29s 79ms/step - loss: 0.6718 - accuracy: 0.7630 - val_loss: 1.1488 - val_accuracy:

In [15]:
# 계층 교차 검증
n_fold = 5  
seed = 42

cv = StratifiedKFold(n_splits = n_fold, shuffle=True, random_state=seed)

# 테스트데이터의 예측값 담을 곳 생성
test_y = np.zeros((test_x.shape[0], 7))

# 조기 종료 옵션 추가
es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                   verbose=1, mode='min', baseline=None, restore_best_weights=True)

for i, (i_trn, i_val) in enumerate(cv.split(train_x, Y_train), 1):
    print(f'training model for CV #{i}')

    model2.fit(train_x[i_trn], 
            to_categorical(Y_train[i_trn]),
            validation_data=(train_x[i_val], to_categorical(Y_train[i_val])),
            epochs=10,
            batch_size=512,
            callbacks=[es])     # 조기 종료 옵션
                      
    test_y += model2.predict(test_x) / n_fold    # 나온 예측값들을 교차 검증 횟수로 나눈다

training model for CV #1
Epoch 1/10
72/72 [==============================] - 18s 255ms/step - loss: 0.6858 - accuracy: 0.7564 - val_loss: 0.6686 - val_accuracy: 0.7609
Epoch 2/10
72/72 [==============================] - 18s 252ms/step - loss: 0.6480 - accuracy: 0.7693 - val_loss: 0.6714 - val_accuracy: 0.7559
Epoch 3/10
72/72 [==============================] - 18s 253ms/step - loss: 0.6318 - accuracy: 0.7724 - val_loss: 0.6819 - val_accuracy: 0.7582
Epoch 4/10
72/72 [==============================] - 19s 257ms/step - loss: 0.6186 - accuracy: 0.7766 - val_loss: 0.6956 - val_accuracy: 0.7540
Restoring model weights from the end of the best epoch.
Epoch 00004: early stopping
training model for CV #2
Epoch 1/10
72/72 [==============================] - 19s 264ms/step - loss: 0.6575 - accuracy: 0.7649 - val_loss: 0.6450 - val_accuracy: 0.7707
Epoch 2/10
72/72 [==============================] - 19s 270ms/step - loss: 0.6364 - accuracy: 0.7701 - val_loss: 0.6569 - val_accuracy: 0.7667
Epoch 3/

In [16]:
# 테스트 데이터의 예측값 확인
test_y

array([[0.02572614, 0.10739847, 0.80665508, ..., 0.00346195, 0.00225981,
        0.00187478],
       [0.07950623, 0.10726345, 0.2322131 , ..., 0.11540738, 0.07888728,
        0.11123841],
       [0.00992322, 0.13168615, 0.84162672, ..., 0.00213506, 0.00088051,
        0.00453327],
       ...,
       [0.01147092, 0.00539301, 0.79948118, ..., 0.1354564 , 0.00741784,
        0.01007499],
       [0.24999254, 0.04271241, 0.32225451, ..., 0.0395082 , 0.03881319,
        0.02081436],
       [0.00618821, 0.00268449, 0.74549425, ..., 0.01032806, 0.00938298,
        0.20711877]])

In [17]:
topic = []
for i in range(len(test_y)):
    topic.append(np.argmax(test_y[i]))

In [18]:
submission

,index,topic_idx
0,45654,0
1,45655,0
2,45656,0
3,45657,0
4,45658,0
...,...,...
9126,54780,0
9127,54781,0
9128,54782,0
9129,54783,0
